In [1]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using CSV
using JLD
using DataFrames
using JSON
using MAT

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2114693016751496712\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-18220768383004287198\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2114693016751496712\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


importing packages...
packages and functions imported
Elapsed time = 15.761101961135864 seconds 
 


In [2]:
### Set parameters

betti_colors = [["#243a4c"] ["#406372"] ["#66939e"] ["#9bc3c6"]]

# Read from config file
config_file = "config101220.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
betti_files = filter(x->!occursin("dsi", x), betti_files)

println("Located the following graph files:")
# for betti_file in betti_files
#     println(betti_file)
# end


# Locate the nametags
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


for nametag in nametags
    namefiles = filter(x->occursin(nametag,x), betti_files)
    if length(namefiles)<12
    println(nametag)
    println(length(namefiles))
    end
end

nametags

Located the following graph files:
IID_70nodes_500reps_101220_forward_bettis.jld
IID_70nodes_500reps_101220_overlap_a01_b03_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a02_b04_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a03_b05_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a04_b06_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a05_b07_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a06_b08_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a07_b09_overlap_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_noiseOnly_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_randomized_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_noiseOnly_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_randomized_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh025_edge604_noiseOnly_bettis.jld
IID_

clique10_70nodes_500reps_10_101220_thresh03_edge725_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh03_edge725_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh03_edge725_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh05_edge1208_noiseOnly_betti

clique20_70nodes_500reps_20_101220_thresh085_edge2053_threshold_bettis.jld
clique20_70nodes_500reps_20_101220_thresh08_edge1932_noiseOnly_bettis.jld
clique20_70nodes_500reps_20_101220_thresh08_edge1932_randomized_threshold_bettis.jld
clique20_70nodes_500reps_20_101220_thresh08_edge1932_threshold_bettis.jld
clique20_70nodes_500reps_20_101220_thresh09_edge2174_noiseOnly_bettis.jld
clique20_70nodes_500reps_20_101220_thresh09_edge2174_randomized_threshold_bettis.jld
clique20_70nodes_500reps_20_101220_thresh09_edge2174_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_forward_bettis.jld
clique25_70nodes_500reps_25_101220_thresh015_edge363_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh015_edge363_randomized_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh015_edge363_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh01_edge242_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh01_edge242_randomized_threshold_bettis.jld
clique25_70node

clique30_70nodes_500reps_30_101220_thresh055_edge1329_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh05_edge1208_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh05_edge1208_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh05_edge1208_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh065_edge1570_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh065_edge1570_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh065_edge1570_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh06_edge1449_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh06_edge1449_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh06_edge1449_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh075_edge1812_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh075_edge1812_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh075_edge1812_threshol

clique_70nodes_500reps_101220_thresh01_edge242_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh025_edge604_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh025_edge604_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh025_edge604_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh02_edge483_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh02_edge483_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh02_edge483_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh035_edge846_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh035_edge846_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh035_edge846_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh03_edge725_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh03_edge725_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh03_edge725_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh045_edge1087_noiseOnly_bettis.

...

wprobTriangle3_70nodes_500reps_03_101220_thresh085_edge2053_threshold_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh08_edge1932_noiseOnly_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh08_edge1932_randomized_threshold_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh08_edge1932_threshold_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh09_edge2174_noiseOnly_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh09_edge2174_randomized_threshold_bettis.jld
wprobTriangle3_70nodes_500reps_03_101220_thresh09_edge2174_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_forward_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh015_edge363_noiseOnly_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh015_edge363_randomized_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh015_edge363_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh01_edge242_noiseOnly_bettis.jld
wprobTriangle4_70nodes_500r

wprobTriangle6_70nodes_500reps_06_101220_thresh035_edge846_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh035_edge846_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh035_edge846_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh03_edge725_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh03_edge725_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh03_edge725_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh045_edge1087_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh045_edge1087_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh045_edge1087_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh04_edge966_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh04_edge966_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh04_edge966_threshold_bettis.jld

wprobTriangle8_70nodes_500reps_08_101220_thresh065_edge1570_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh065_edge1570_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh06_edge1449_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh06_edge1449_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh06_edge1449_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh075_edge1812_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh075_edge1812_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh075_edge1812_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh07_edge1691_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh07_edge1691_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh07_edge1691_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh085_edge2053_noiseOnly_b

wprobTriangle9_70nodes_500reps_09_101220_thresh09_edge2174_randomized_threshold_bettis.jld
wprobTriangle9_70nodes_500reps_09_101220_thresh09_edge2174_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_forward_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh015_edge363_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh015_edge363_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh015_edge363_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh01_edge242_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh01_edge242_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh01_edge242_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh025_edge604_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh025_edge604_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh025_edge604_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_

59-element Array{Any,1}:
 "forward"                                
 "overlap_a01_b03_overlap"                
 "overlap_a02_b04_overlap"                
 "overlap_a03_b05_overlap"                
 "overlap_a04_b06_overlap"                
 "overlap_a05_b07_overlap"                
 "overlap_a06_b08_overlap"                
 "overlap_a07_b09_overlap"                
 "thresh015_edge363_noiseOnly"            
 "thresh015_edge363_randomized_threshold" 
 "thresh015_edge363_threshold"            
 "thresh01_edge242_noiseOnly"             
 "thresh01_edge242_randomized_threshold"  
 ⋮                                        
 "thresh07_edge1691_noiseOnly"            
 "thresh07_edge1691_randomized_threshold" 
 "thresh07_edge1691_threshold"            
 "thresh085_edge2053_noiseOnly"           
 "thresh085_edge2053_randomized_threshold"
 "thresh085_edge2053_threshold"           
 "thresh08_edge1932_noiseOnly"            
 "thresh08_edge1932_randomized_threshold" 
 "thresh08_edge1932_threshold

In [3]:

####### Writing for JSON
using JSON

## Filter to include JUST the threshold ones
thresh_files = filter(x -> occursin("threshold",x),betti_files)
thresh_files = filter(x -> !occursin("randomized",x),thresh_files)
thresh_files = filter(x -> !occursin("clique",x),thresh_files)
thresh_files = filter(x -> !occursin("Triangle",x),thresh_files)
thresh_files = filter(x -> !occursin("noiseOnly",x),thresh_files)

## Filter to only include every kth data point - for faster visualization loading
k = 4


model_names = []
for thresh_file in thresh_files
    name = split(thresh_file, "_")[1]
    model_names = [model_names; name]
end

unique!(model_names)
betti_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform","dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ];

big_dict = Dict()
df = DataFrame(edge = collect(1:binomial(NNODES,2)))

for (i,model_name) in enumerate(model_names)



    model_dict = Dict()

    # Run through all the threshold files and extract Betti curves

    model_thresh_files = filter(x->startswith(x,model_name), thresh_files)

    

    for model_thresh_file in model_thresh_files
        

        rho_dict = Dict()

        bettisArray = load("../processed_data/results/70nodes/$(model_thresh_file)","bettisArray")
        bettisArrayAvg = dropdims(mean(bettisArray, dims=1), dims=1)
        bettisArrayStd = dropdims(std(bettisArray,dims=1), dims=1)
        
        # Keep only ever kth datapoint
        nEdges = size(bettisArray)[2]
        keep_data = collect(1:k:nEdges)

        
        # Extract the rho value or edge number and add to dictionary
        thresh_edge = parse(Int,split(split(model_thresh_file,"edge")[2], "_")[1])
        rho = replace(split(split(model_thresh_file,"_edge")[1], "_thresh")[2], "."=> "")
#         println(thresh_edge)


        for dim in 1:MAXDIM
            rho_dict["dim$(dim)"] = bettisArrayAvg[keep_data,dim]
#             rho_dict["std$(dim)"] = bettisArrayStd[keep_data, dim]
            df[!, "$(model_name)_thresh$(rho)_dim$(dim)"] = bettisArrayAvg[:, dim]
        end




        model_dict["$(thresh_edge)"] = rho_dict

    end


    big_dict["$(betti_names[i])"] = model_dict
#     big_dict["$(model_name)"] = model_dict
    
    
    println("finished $(model_name)")
    
end

big_dict





# df

finished IID
finished assortative
finished coreperiphery
finished cosineGeometric
finished disassortative
finished discreteUniformConf
finished dotProduct
finished geometricConf
finished randomGeom
finished ringLattice
finished rmsd
finished sqdEuclidean


Dict{Any,Any} with 12 entries:
  "discreteUniform"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "cosineGeometric"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "dotProduct"       => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "geometricConf"    => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "disassortative"   => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "assortative"      => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "rmsd"             => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "squaredEuclidean" => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "ringLattice"      => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "IID"              => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "randomGeometric"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "coreperiphery"    => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 

In [4]:
big_dict["IID"]["966"]

Dict{Any,Any} with 4 entries:
  "dim2" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0…
  "dim1" => [0.0, 0.0, 0.0, 0.006, 0.01, 0.024, 0.048, 0.108, 0.188, 0.348  …  …
  "dim3" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0…
  "dim4" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0…

In [5]:
## Write to json
open("../templates/main_k$(k).json", "w") do f
    JSON.print(f, big_dict)
end
println("done saving")

done saving


## Loading and wrapping classification mat files


In [6]:
model_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform",
    "dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ]
mat_dir = "../processed_data/classification/"
mat_files = readdir(mat_dir)
mat_files = filter(x -> occursin(".mat",x), mat_files)

experiment = "prenoise"
mat_files = filter(x-> occursin(experiment,x), mat_files)
mat_files = filter(x-> !occursin("NO",x), mat_files)
mat_files = filter(x-> !occursin("Rand",x), mat_files)

17-element Array{String,1}:
 "matrix_prenoise_0.1.mat" 
 "matrix_prenoise_0.15.mat"
 "matrix_prenoise_0.2.mat" 
 "matrix_prenoise_0.25.mat"
 "matrix_prenoise_0.3.mat" 
 "matrix_prenoise_0.35.mat"
 "matrix_prenoise_0.4.mat" 
 "matrix_prenoise_0.45.mat"
 "matrix_prenoise_0.5.mat" 
 "matrix_prenoise_0.55.mat"
 "matrix_prenoise_0.6.mat" 
 "matrix_prenoise_0.65.mat"
 "matrix_prenoise_0.7.mat" 
 "matrix_prenoise_0.75.mat"
 "matrix_prenoise_0.8.mat" 
 "matrix_prenoise_0.85.mat"
 "matrix_prenoise_0.9.mat" 

In [ ]:
# Make a n_threhsolds x n_models^2 matrix
#### MARK FOR DELETION
mat1 = matread("$(mat_dir)/$(mat_files[1])")
mat1["C"]

In [ ]:
#### MARK FOR DELETION
col_names = ["$(a)_$(b)" for a in model_names for b in model_names]

In [ ]:
#### MARK FOR DELETION
df = DataFrame(mat1["C"])
rename!(df,Symbol.(model_names))
df.true_class = model_names
df

In [ ]:
#### MARK FOR DELETION
df2 = stack(df,1:12)
rename!(df2, [:trueClass, :predictedClass, :accuracy])

In [ ]:
#### MARK FOR DELETION
df2.truePredicted = ["$(b)_$(a)" for a in model_names for b in model_names]
df2

In [ ]:
#### MARK FOR DELETION
# Get the two vectors we care about
df3 = DataFrame(TP = df2.truePredicted)


In [7]:
# Write the first to df3, then repeat

mat1 = matread("$(mat_dir)/$(mat_files[1])")
mat1["C"]
col_names = ["$(a)_$(b)" for a in model_names for b in model_names]
df = DataFrame(mat1["C"])
rename!(df,Symbol.(model_names))
df.true_class = model_names
df2 = stack(df,1:12)
rename!(df2, [:trueClass, :predictedClass, :accuracy])
df2.truePredicted = ["$(b)_$(a)" for a in model_names for b in model_names]

# Get the two vectors we care about
df3 = DataFrame(TP = df2.truePredicted)

# repeat

for mat_file in mat_files
    mat = matread("$(mat_dir)/$(mat_file)")["C"]
    thresh = "thresh$(replace(split(split(mat_file, ".mat")[1], "_")[3], "."=>""))"
    df = stack(DataFrame(mat),1:12)
    println(thresh)
    df3[!, thresh] = df.value
end
df3

thresh01
thresh015
thresh02
thresh025
thresh03
thresh035
thresh04
thresh045
thresh05
thresh055
thresh06
thresh065
thresh07
thresh075
thresh08
thresh085
thresh09


,TP,thresh01,thresh015,thresh02,thresh025,thresh03
,String,Float64,Float64,Float64,Float64,Float64
1,IID_IID,154.0,202.0,172.0,249.0,238.0
2,assortative_IID,0.0,0.0,0.0,0.0,0.0
3,coreperiphery_IID,172.0,3.0,0.0,0.0,0.0
4,cosineGeometric_IID,0.0,0.0,0.0,0.0,0.0
5,disassortative_IID,64.0,74.0,32.0,250.0,13.0
6,discreteUniform_IID,0.0,0.0,0.0,0.0,0.0
7,dotProduct_IID,0.0,0.0,0.0,0.0,0.0
8,geometricConf_IID,0.0,0.0,0.0,0.0,0.0
9,randomGeometric_IID,0.0,0.0,0.0,0.0,0.0


In [8]:
CSV.write("../templates/classification_prenoise_101220.csv",df3)

"../templates/classification_prenoise_101220.csv"